In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import JavascriptException
from bs4 import BeautifulSoup
import time
import json
import requests
import pandas as pd
import numpy as np

titles=[]
years=[]
imdb_ratings=[]
metascores=[]
no_of_votes=[]
#gross=[]
genres=[]
runtime=[]
certificates=[]
imgurl=[]
plot=[]

links=[]

fulltime_start=time.time()

for yr in range(1960,2025):
    seltime_start=time.time()  
    chrome_options = webdriver.ChromeOptions()
    chrome_options.set_capability("goog:loggingPrefs", {"performance": "ALL"})

    driver = webdriver.Chrome(options=chrome_options)
    url = f"https://www.imdb.com/search/title/?title_type=feature&release_date={yr}-01-01,{yr}-12-31&sort=num_votes,desc&count=250"
    driver.get(url) 
    #time.sleep(10)
    
    #time.sleep(40)
    try:
        #wait = WebDriverWait(driver, 10)
        
        #initial_items = driver.find_elements(By.CSS_SELECTOR, ".ipc-metadata-list-summary-item")
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".ipc-see-more__button"))
        )

        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
        time.sleep(1)

        try:
            button.click()
        except Exception:
            driver.execute_script("arguments[0].click();", button)

        #button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".ipc-see-more__button")))
        # Scroll to the button
        #driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", button)
        # Optional: Wait for scroll and rendering
        #time.sleep(1)
        #time.sleep(5)
        #button.click()
        #wait.until(lambda d: len(d.find_elements(By.CSS_SELECTOR, ".ipc-metadata-list-summary-item")) > len(initial_items))
        html = driver.page_source
    # Step 5: Parse with BeautifulSoup
        soup1 = BeautifulSoup(html, 'html.parser')
        js_res=json.loads(soup1.find("script",{"id":"__NEXT_DATA__"}).text)
        for edge in js_res['props']['pageProps']['searchResults']['titleResults']['titleListItems']:
            titles.append(edge['originalTitleText'])
            if 'genres' in edge:
                genres.append(edge['genres'])
            else:
                genres.append(np.nan)
            if 'primaryImage' in edge and 'url' in edge['primaryImage']:
                imgurl.append(edge['primaryImage']['url'])
            else:
                imgurl.append(np.nan)
            if 'metascore' in edge:
                metascores.append(edge['metascore'])
            else:
                metascores.append(np.nan)
            years.append(yr)
            #year.append(edge['releaseDate']['year'])
            imdb_ratings.append(edge['ratingSummary']['aggregateRating'])
            no_of_votes.append(edge['ratingSummary']['voteCount'])
            if 'certificate' in edge:
                certificates.append(edge['certificate'])
            else:
                certificates.append(np.nan)
            if 'runtime' in edge:
                runtime.append(edge['runtime'])
            else:
                runtime.append(np.nan)
            if 'plot' in edge:
                plot.append(edge['plot'])
            else:
                plot.append(np.nan)

        logs = driver.get_log("performance")
        xhr_requests = []
        for entry in logs:
            log = json.loads(entry["message"])["message"]
            if log["method"] == "Network.requestWillBeSent" and "request" in log["params"]:
                request_url = log["params"]["request"]["url"]
                request_type = log["params"]["request"]["method"]
                if request_type in ["POST", "GET"] and (
                    "xhr" in log["params"]["type"] or "Fetch" in log["params"]["type"]
                ):
                    xhr_requests.append(request_url)
                    
        for xhr in xhr_requests:
            if 'https://caching.graphql.imdb.com/?operationName=AdvancedTitleSearch' in xhr:
                links.append(xhr)
                
        driver.quit()
                
    except JavascriptException as e:
        print("JavaScript error:", e)
    except IndexError:
        print("No button found.")
    if yr%20==0:
        movies=pd.DataFrame({
            'title':titles,
            'year':years,
            'imdb_rating':imdb_ratings,
            'metascore':metascores,
            'votes':no_of_votes,
            #'Box_office':gross,
            'genre':genres,
            'runtime':runtime,
            'certificates':certificates,
            'img_url':imgurl,
            'plot':plot
        }) 
        movies.to_csv(f'movies_F_{yr}.csv')
        titles=[]
        years=[]
        imdb_ratings=[]
        metascores=[]
        no_of_votes=[]
        #gross=[]
        genres=[]
        runtime=[]
        certificates=[]
        imgurl=[]
        plot=[]
        movies=pd.DataFrame()
        

In [25]:
movies=pd.DataFrame({
    'title':titles,
    'year':years,
    'imdb_rating':imdb_ratings,
    'metascore':metascores,
    'votes':no_of_votes,
    #'Box_office':gross,
    'genre':genres,
    'runtime':runtime,
    'certificates':certificates,
    'img_url':imgurl,
    'plot':plot
}) 
movies.to_csv(f'movies_F_{yr}.csv')

In [26]:
import requests
titles=[]
years=[]
imdb_ratings=[]
metascores=[]
no_of_votes=[]
#gross=[]
genres=[]
runtime=[]
certificates=[]
imgurl=[]
plot=[]
#result=[]
yr=1960
for url in links:
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36",
        "Content-Type": "application/json"     
    }
    res = requests.get(url,headers=headers)
    print(res)
    #print(res.text)
    for edge in json.loads(res.text)['data']['advancedTitleSearch']['edges']:
        titles.append(edge['node']['title']['originalTitleText']['text'])
        #titles.append(moviename)
        years.append(yr)
        imdb_ratings.append(edge['node']['title']['ratingsSummary']['aggregateRating'])

        if edge['node']['title']['certificate']:
            certificates.append(edge['node']['title']['certificate']['rating'])
        else:
            certificates.append(np.nan)

        if edge['node']['title']['metacritic']:
            metascores.append(edge['node']['title']['metacritic']['metascore']['score'])
        else:
            metascores.append(np.nan)

        genre1=[]
        if edge['node']['title']['titleGenres']:
            for i in edge['node']['title']['titleGenres']['genres']:
                genre1.append(i['genre']['text'])
            genres.append(genre1)
        else:
            genres.append(genre1)

        no_of_votes.append(edge['node']['title']['ratingsSummary']['voteCount'])

        if edge['node']['title']['primaryImage'] and 'url' in edge['node']['title']['primaryImage']:
            imgurl.append(edge['node']['title']['primaryImage']['url'])
        else:
            imgurl.append(np.nan)

        if edge['node']['title']['runtime']:
            runtime.append(edge['node']['title']['runtime']['seconds'])
        else:
            runtime.append(np.nan)
            
        if edge['node']['title']['plot'] and 'plotText' in edge['node']['title']['plot']:
            plot.append(edge['node']['title']['plot']['plotText']['plainText'])
        else:
            plot.append(np.nan)
                                
    if yr%20==0:
        movies=pd.DataFrame({
            'title':titles,
            'year':years,
            'imdb_rating':imdb_ratings,
            'metascore':metascores,
            'votes':no_of_votes,
            #'Box_office':gross,
            'genre':genres,
            'runtime':runtime,
            'certificates':certificates,
            'img_url':imgurl,
            'plot':plot
        }) 
        movies.to_csv(f'movies_S_{yr}.csv')
        titles=[]
        years=[]
        imdb_ratings=[]
        metascores=[]
        no_of_votes=[]
        #gross=[]
        genres=[]
        runtime=[]
        certificates=[]
        imgurl=[]
        plot=[]
    yr+=1

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [28]:
movies=pd.DataFrame({
    'title':titles,
    'year':years,
    'imdb_rating':imdb_ratings,
    'metascore':metascores,
    'votes':no_of_votes,
    #'Box_office':gross,
    'genre':genres,
    'runtime':runtime,
    'certificates':certificates,
    'img_url':imgurl,
    'plot':plot
}) 
movies.to_csv(f'movies_S_{yr}.csv')

In [5]:
res.text

'{"errors":[{"message":"PersistedQueryNotFound","extensions":{"code":"PERSISTED_QUERY_NOT_FOUND","errorType":"CLIENT","isRetryable":false}}],"extensions":{"disclaimer":"Public, commercial, and/or non-private use of the IMDb data provided by this API is not allowed. For limited non-commercial use of IMDb data and the associated requirements see https://help.imdb.com/article/imdb/general-information/can-i-use-imdb-data-in-my-software/G5JTRESSHJBBHTGX#"}}\n'

In [8]:
movies

,title,year,imdb_rating,metascore,votes,genre,runtime,certificates,img_url
0,The Starling,2021,6.4,31.0,16305,"[Comedy, Drama]",6120,PG-13,https://m.media-amazon.com/images/M/MV5BYmYwMz...
1,Rang song,2021,6.5,NaN,16168,"[Drama, Horror, Mystery]",7800,Not Rated,https://m.media-amazon.com/images/M/MV5BMzliMj...
2,Peter Rabbit 2: The Runaway,2021,6.2,43.0,16076,"[Animation, Adventure, Comedy]",5580,PG,https://m.media-amazon.com/images/M/MV5BODA3ZT...
3,The Alpinist,2021,7.9,67.0,16068,"[Documentary, Biography]",5520,PG-13,https://m.media-amazon.com/images/M/MV5BYmU4OG...
4,Val,2021,7.6,73.0,15870,"[Documentary, Biography]",6540,R,https://m.media-amazon.com/images/M/MV5BZWUxMW...
...,...,...,...,...,...,...,...,...,...
995,Martha,2024,7.2,69.0,4215,[Documentary],6900,R,https://m.media-amazon.com/images/M/MV5BNDZkNz...
996,Malaikottai Vaaliban,2024,6.0,NaN,4197,"[Action, Drama, Fantasy]",9360,NaN,https://m.media-amazon.com/images/M/MV5BYzYzNz...
997,Gowri,2024,5.4,NaN,4184,[Family],7800,NaN,https://m.media-amazon.com/images/M/MV5BNzdkOG...
998,Einstein and the Bomb,2024,6.2,NaN,4149,"[Documentary, History]",4560,TV-MA,https://m.media-amazon.com/images/M/MV5BODc2MD...


In [10]:
edge['node']['title']['plot']['plotText']['plainText']

'In this true-crime documentary, three guys exploit the freewheeling cryptocurrency market to scam millions from investors and bankroll lavish lifestyles.'

In [15]:
edge['plot']

"Tula, a 31-year-old single woman whose sister has just died, decides to live in the home of her bank-employee brother-in-law Ramiro and his two children. She takes control of their lives, playing wife/mother without the sexual commitments or maternal responsibilities. Ramiro likes her because she adores his children, but she rejects his courtship. Desperate Ramiro tries to rape her and the priest advises her to marry the man. Tula insists on maintaining platonic relations and decides to take them to her village and, in the presence of relatives, redirect Ramiro's feelings to the memory of her deceased sister. Ramiro rapes Tula's young cousin Juanita at the earliest opportunity, but the rest of the family does not know until months later when it's obvious that she's pregnant. Ramiro is forced to marry her, and he takes her and the children back to the city."

In [24]:
len(links)

65